In [5]:
# import sympy
%display typeset

In [6]:
var(' S E I R gamma mu alpha beta')

(S, E, I, R, gamma, mu, alpha, beta)

In [12]:
dsdt = -beta*S*I
dedt = beta*S*I - mu*E
didt = mu*E - alpha*I - gamma*I
drdt = alpha*I

In [13]:
solve([dsdt,dedt,didt,drdt],[S,E,I,R])

[[S == r1, E == 0, I == 0, R == r2]]

In [5]:
jack=jacobian([dsdt,dedt,didt,drdt],[S,E,I,R])
jack

[       -I*beta              0        -S*beta              0]
[        I*beta            -mu         S*beta              0]
[             0             mu -alpha - gamma              0]
[             0              0          alpha              0]

In [6]:
cp = jack.characteristic_polynomial()
cp

x^4 + (I*beta + alpha + gamma + mu)*x^3 + (I*alpha*beta + I*beta*gamma + I*beta*mu - S*beta*mu + alpha*mu + gamma*mu)*x^2 + (I*alpha*beta*mu + I*beta*gamma*mu)*x

In [8]:
jack.eigenvalues();

compartimentos com infectados: E e I

In [7]:
F_cal = matrix([[beta*S*I],[0]])

In [8]:
V_cal_neg = matrix([[mu*E],[alpha*I + gamma*I]])

In [9]:
V_cal_pos = matrix([[beta*S*I],[mu*E]])

In [10]:
V_cal = V_cal_neg - V_cal_pos
V_cal

[        -I*S*beta + E*mu]
[I*alpha + I*gamma - E*mu]

In [11]:
F = jacobian(F_cal(S=1),[E,I])
F

[   0 beta]
[   0    0]

In [12]:
V = jacobian(V_cal(S=1),[E,I])
V

[           mu         -beta]
[          -mu alpha + gamma]

In [13]:
M = F*V.inverse()
M

[beta/(alpha - beta + gamma) beta/(alpha - beta + gamma)]
[                          0                           0]

In [14]:
M=M.simplify_full()
M

[beta/(alpha - beta + gamma) beta/(alpha - beta + gamma)]
[                          0                           0]

In [23]:
M.eigenvalues()


[beta/(alpha - beta + gamma), 0]

In [15]:
R0=M[0,0].simplify_full()
R0

beta/(alpha - beta + gamma)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
!pip install pandas

  Using cached https://files.pythonhosted.org/packages/fb/e4/828bb9c2474ff6016e5ce96a78220d485436d5468c23068f4f6c2eb9cff8/pandas-1.1.5.tar.gz
  Installing build dependencies ... done
  Running setup.py install for pandas ... done


In [3]:
import pandas as pd

In [7]:
def modelo(xs,t,ps):
    try:
        gamma = ps['gamma'].value 
        mu = ps['mu'].value
        alpha = ps['alpha'].value
        beta = ps['beta'].value
    except:
        gamma, mu, alpha, beta = ps
        
    
    S, E, I, R = xs
    
    return[
        -beta*S*I, 
        beta*S*I - mu*E,
        mu*E - alpha*I - gamma*I,
        alpha*I     
    ]

#então, estou com um problema nessa função engine porque essa função solve não funciona como eu quero que ela funcione
#então eu vou deixar isso pra depois
def engine(t, x0, ps):
    S = 1 - np.sum(x0)
    x0 = np.insert(x0, 0, S)
    x = solve(modelo, x0, t, args = (ps,))
    return x

In [15]:
t = np.linspace(0,50,200)
x0 = [10e-6,0,0]


gamma = 1
mu = 1
alpha = 1
beta = 5

ps = np.array((gamma, mu, alpha, beta))
sol = engine(t,x0,ps)

TypeError: array([9.9999e-01, 1.0000e-05, 0.0000e+00, 0.0000e+00]) is not a valid variable.

In [10]:
!pip install lmfit

    100% |████████████████████████████████| 266kB 810kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 768kB/s ta 0:00:01███████████                    | 20kB 7.3MB/s eta 0:00:01
    100% |████████████████████████████████| 256kB 827kB/s ta 0:00:01
  Running setup.py install for asteval ... done
  Running setup.py install for lmfit ... done


In [11]:
from lmfit import *

/opt/sagemath-9.1/local/lib/python3.7/site-packages/past/types/oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [12]:
def loss(ps, ts, data):
    x0 = ps['E'].value,ps['I'].value,ps['R'].value
    f = engine(ts,x0, ps)
    s = f[:,1]
    return sol[:,1]

In [16]:
dados = sol[:,1] + np.random.normal(scale = 0.01 , size = np.shape(sol[:,1]))
plt.figure(figsize=(8, 6))

#essa célula aqui é de criar dados aleatórios só pra testar se as nossas funções estavam funcionando, depois essa variável
#dados tem que ser substituida pelos dados de verdade, que provavelmente temos o csv disponível pra usar
plt.plot(t, dados)
plt.plot(t, sol[:,1])

NameError: name 'sol' is not defined

In [13]:
params = Parameters()

params.add('E', value=0.0, min=0, max=1,vary= False)
params.add('R', value=0.0, min=0, max=1, vary= False)
params.add('I', value=10e-6, min=0, max=10e-4, vary= True)
params.add('gamma', value=0, min=0, max=4)
params.add('mu', value=0, min=0, max=1)
params.add('alpha', value=0, min=0, max=1)
params.add('beta', value=0, min=0, max=1)

final = minimize(loss, params, args = (t, dados), method = 'leastsq')

NameError: name 'dados' is not defined